<a href="https://colab.research.google.com/github/MMatheus-Silva/noticias_web/blob/master/Trabalho_BigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 50.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.0 MB/s eta 0:00

In [34]:
import requests
import json
import pandas as pd
import pycaret
import re
pycaret.__version__


'3.3.2'

In [35]:
# Ler o arquivo CSV
df = pd.read_csv('/mercadolivreconsole.csv', encoding='utf-8')

#df.rename(columns={'Descrição': 'descricao'}, inplace=True)

print("Colunas disponíveis:", df.columns)

# Cabeçalhos da requisição
headers = {
    "Content-Type": "application/json"
}


Colunas disponíveis: Index(['ID', 'Título', 'Condição', 'Link', 'Tipo de Anúncio', 'Modo de Compra',
       'Site ID', 'Categoria ID', 'Domínio ID', 'Moeda', 'Preço',
       'Preço Original', 'Quantidade Disponível', 'Quantidade Vendida',
       'ID da Loja Oficial', 'Aceita MercadoPago', 'Envio Grátis',
       'Tipo de Logística', 'Modo de Envio', 'Data de Término', 'Vendedor ID',
       'Vendedor Apelido', 'Modelo', 'Marca', 'Condição do Item', 'Submodelo',
       'Quantidade de Parcelas', 'Valor da Parcela', 'Taxa de Juros',
       'ID de Preço Diferencial', 'Tags de Envio', 'Variação de Filtros',
       'Descrição'],
      dtype='object')


In [46]:

# Função para verificar defeitos
def verificar_defeito(descricao):
    if isinstance(descricao, str):  # Verifica se é uma string
        padrao_defeito = r'defeito|avaria|quebrado\(a\)|rachado\(a\)|arranhado\(a\)|problema|danificada\(o\)'
        return bool(re.search(padrao_defeito, descricao, re.IGNORECASE))
    return False  # Retorna False se não for string

# Função para verificar modificação
def verificar_modificacao(descricao):
    if isinstance(descricao, str):  # Verifica se é uma string
        padrao_modificacao = r'modificado\(a\)|modificação'
        return bool(re.search(padrao_modificacao, descricao, re.IGNORECASE))
    return False  # Retorna False se não for string

# Criando novas colunas com os valores booleanos
df['Defeito'] = df['Descrição'].apply(verificar_defeito)
df['Modificacao'] = df['Descrição'].apply(verificar_modificacao)

# Filtrando as linhas onde Defeito ou Modificacao é verdadeiro
resultado_filtrado = df[(df['Defeito'] == True) | (df['Modificacao'] == True)]

# Exibindo o DataFrame resultante
colunas_especificas = resultado_filtrado[['ID', 'Título', 'Descrição', 'Preço', 'Defeito', 'Modificacao']]
print(colunas_especificas.head())




               ID                                             Título  \
16  MLB3830888405                Placa Mãe Com Defeito Xbox Series S   
20  MLB4959053208  Console Playstation 3 Slim 120gb Na Caixa Com ...   
36  MLB4814063470  Console Playstation 3 Ps3 Super Slim Original ...   
40  MLB4995537830        Console Nintendo 64 Jabuticaba Com Controle   
46  MLB3833842923     Playstation 3 Fat Retrocompatível Ylod Lacrado   

                                            Descrição Preço  Defeito  \
16  ATENÇÃO: PRODUTO COM DEFEITO, PARA RETIRADA DE...   399     True   
20  Bem-vindo à Kanto dos Jogos - Sua Melhor Escol...  1299     True   
36  ¬ ATENÇÃO:\n\n- Console original do Playstatio...   780     True   
40  Bem-vindo à Kanto dos Jogos - Sua Melhor Escol...  1799     True   
46  ATENÇÃO: Console com problema dos 3 bips, desl...   599     True   

    Modificacao  
16        False  
20        False  
36        False  
40        False  
46        False  


In [47]:
import re

# Função para verificar defeitos
def verificar_defeito(descricao):
    if isinstance(descricao, str):  # Verifica se é uma string
        padrao_defeito = r'defeito|avaria|quebrado\(a\)|rachado\(a\)|arranhado\(a\)|problema|danificada\(o\)'
        match = re.search(padrao_defeito, descricao, re.IGNORECASE)
        if match:
            print(f"Encontrado: {match.group()} na descrição: {descricao}")  # Para depuração
            return True
    return False  # Retorna False se não for string

# Testando a função com sua descrição
descricao = """Bem-vindo à Kanto dos Jogos - Sua Melhor Escolha no Mercado Livre!

Antes de fazer sua compra, não hesite em esclarecer todas as suas dúvidas. Estamos aqui para garantir a sua satisfação!

Características do Produto:

Produto Original: Garantimos que todos os nossos produtos são genuínos e autênticos.
Ótimo estado
Funcionando Perfeitamente: Testamos minuciosamente todos os jogos e consoles para garantir que funcionem sem problemas.
- Acompanha controle, cabo AV, cabo HDMI, jogos e cabo de força.

Vantagens de Comprar com a Kanto dos Jogos:

Emissão de Nota Fiscal: Oferecemos segurança e transparência em todas as suas compras.
Garantia de 3 Meses: Todos os jogos e consoles incluem uma garantia de 3 meses para sua tranquilidade.
Suporte Pós-Venda: Estamos sempre disponíveis para ajudar com quaisquer problemas ou dúvidas que possam surgir após a compra.
Envio Rápido: Pedidos feitos até as 10:00 são enviados no mesmo dia, garantindo que você receba seus produtos o mais rápido possível.
Brindes Especiais: Como um gesto de apreço, também incluímos brindes surpresa em suas encomendas para tornar sua experiência ainda mais especial!

Nossa Reputação no Mercado Livre:
Trabalhamos arduamente para manter nossa posição como Mercado Líder Gold, um dos melhores vendedores do site. Você pode confiar na Kanto dos Jogos para oferecer produtos de alta qualidade, excelente atendimento ao cliente e uma experiência de compra excepcional.

Não hesite em escolher a Kanto dos Jogos para todas as suas necessidades de jogos e consoles. Estamos aqui para atendê-lo e garantir que você tenha a melhor experiência de compra possível. Obrigado por escolher a Kanto dos Jogos!!"""

# Teste
print(verificar_defeito(descricao))  # Deve retornar False


Encontrado: problema na descrição: Bem-vindo à Kanto dos Jogos - Sua Melhor Escolha no Mercado Livre! 

Antes de fazer sua compra, não hesite em esclarecer todas as suas dúvidas. Estamos aqui para garantir a sua satisfação!

Características do Produto:

Produto Original: Garantimos que todos os nossos produtos são genuínos e autênticos.
Ótimo estado
Funcionando Perfeitamente: Testamos minuciosamente todos os jogos e consoles para garantir que funcionem sem problemas.
- Acompanha controle, cabo AV, cabo HDMI, jogos e cabo de força.

Vantagens de Comprar com a Kanto dos Jogos:

Emissão de Nota Fiscal: Oferecemos segurança e transparência em todas as suas compras.
Garantia de 3 Meses: Todos os jogos e consoles incluem uma garantia de 3 meses para sua tranquilidade.
Suporte Pós-Venda: Estamos sempre disponíveis para ajudar com quaisquer problemas ou dúvidas que possam surgir após a compra.
Envio Rápido: Pedidos feitos até as 10:00 são enviados no mesmo dia, garantindo que você receba seus 

In [30]:
# Supondo que você tenha um DataFrame df já criado com a coluna 'Descrição'

def analisar_descricao(descricao):
    # Dicionário com as frases-chave e suas ações
    padroes = {
        "defeito": "analise se o console tem defeito.",
        "defeito na tela": "analise se o console tem defeito na tela.",
        "defeito no hd": "analise se o console tem defeito no hd.",
        "defeito na placa": "analise se o console tem defeito na placa.",
        "defeito na fonte": "analise se o console tem defeito na fonte.",
        "defeito na carcaça": "analise se o console tem defeito na carcaça.",
        "modificado": "analise se o console é modificado.",
        "destravado": "analise se o console é destravado.",
        "itens inclusos": "analise se o console tem itens inclusos.",
        "jogos inclusos": "analise se o console tem jogos inclusos.",
        "quantidade de jogos": "responda a quantidade de jogos inclusos.",
        "quantidade de controles": "responda a quantidade de controles inclusos.",
        "quantidade de jogos": re.compile(r"quantidade de jogos: (\d+)")
    }

    resultados = {}
    if isinstance(descricao, str):
        for padrao, acao in padroes.items():
            if padrao in descricao:
                if isinstance(acao, str):
                    resultados[padrao] = acao
                else:  # Se for uma expressão regular
                    match = acao.search(descricao)
                    if match:
                        resultados[padrao] = match.group(1)
    return resultados  # Garante que sempre retorna um dicionário, mesmo que vazio

# Aplicar a função ao DataFrame
df['analise'] = df['Descrição'].apply(analisar_descricao)

# Extrair informações específicas para novas colunas (exemplo para "quantidade de jogos")
df['qtd_jogos'] = df['analise'].apply(lambda x: x.get('quantidade de jogos', 0) if x else 0)

colunas_especificas = df[['Título', 'Descrição', 'Preço']]
print(colunas_especificas.head())

#print(df.head())  # Visualizar as primeiras linhas com os resultados da análise

                                              Título  \
0    Nvidia Shield Portable Console Android Portátil   
1  Playstation Pro 4k Edição Especial Destiny 2,c...   
2                Sony Ps Vita Slim Standard Seminovo   
3       Headset Astro A20 Wireless Xbox One Seminovo   
4                         Console - Nokia N-gage (1)   

                                           Descrição   Preço  
0  Nvidia shield portatil. Excelente estado. Está...    1200  
1  PLAYSTATION 4 PRO EDIÇAO DESTINY 2\n\nAPARELHO...    2699  
2  PS Vita Slim Seminovo na cor preta. \nGarantia...     950  
3  -Somos Loja física com mais de 13 anos no merc...     399  
4  ---------------------------------\nLOJA GAMEFI...  3569,9  


In [10]:
# Acumular resultados
resultados = []

# Definir as perguntas
perguntas = [
    ("defeito", "analise se o console tem defeito."),
    ("defeito na tela", "analise se o console tem defeito na tela."),
    ("defeito no hd", "analise se o console tem defeito no hd."),
    ("defeito na placa", "analise se o console tem defeito na placa."),
    ("defeito na fonte", "analise se o console tem defeito na fonte."),
    ("defeito na carcaça", "analise se o console tem defeito na carcaça."),
    ("modificado", "analise se o console é modificado."),
    ("destravado", "analise se o console é destravado."),
    ("itens inclusos", "analise se o console tem itens inclusos."),
    ("jogos inclusos", "analise se o console tem jogos inclusos."),
    ("quantidade de jogos", "responda a quantidade de jogos inclusos."),
    ("quantidade de controles", "responda a quantidade de controles inclusos."),
]

In [11]:
# URL da API local do Ollama
url = "http://localhost:11434/api/generate"

for descricao in df['Descrição']:
    response_acumulator = ""

    for chave, pergunta in perguntas:
        payload = {
            "model": "llama3.1:8b",
            "prompt": f"Com base na seguinte descrição: {descricao}, separada por vírgulas, {pergunta} Se o console tiver qualquer defeito, responda apenas com o número 1. Se o console não tiver defeito ou se essa informação não estiver na descrição, responda apenas com o número 0. Não inclua mais nada na resposta, apenas o número."
        }

        # Faz a requisição POST para a API do Ollama
        response = requests.post(url, headers=headers, data=json.dumps(payload))

        # Verifica se a requisição foi bem-sucedida
        if response.status_code == 200:
            for line in response.iter_lines():
                try:
                    # Decodifica cada linha como JSON
                    data = json.loads(line.decode('utf-8'))
                    response_acumulator += data['response']
                    print(data['response'])  # Imprime a resposta
                except json.JSONDecodeError as e:
                    print(f"Erro ao decodificar JSON: {e}")
                    print(f"Linha com erro: {line}")
        else:
            print(f"Erro na requisição: {response.status_code}")

    # Armazena o resultado acumulado para cada descrição
    resultados.append(response_acumulator)

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a9a0d1e6440>: Failed to establish a new connection: [Errno 111] Connection refused'))